[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/morganmcg1/deep-research-bot/blob/main/notebooks/01_simple_agent.ipynb)


# Introduction to Building LLM Agents with Tools and Tracing

<!--- @wandbcode{fc-london-workshop-2025} -->

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Running the agent on a multi-step task.

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!uv pip install -qqq git+https://github.com/morganmcg1/deep-research-bot.git
```


In [1]:
!uv pip install -qqq git+https://github.com/morganmcg1/deep-research-bot.git

In [2]:
# Global Configuration & Setup
import os
import inspect
import json
import weave # Must import weave before litellm for auto-patching
import openai
from enum import Enum
from pydantic import BaseModel, Field
from rich.panel import Panel
from rich.markdown import Markdown
from rich.console import Console as RichConsole
from exa_py import Exa
from typing import Any, Callable, get_type_hints

### Add your API Keys
We'll need a wandb api key and an exa api key

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

False

In [4]:
# or

os.environ["WANDB_API_KEY"] = ""
os.environ["EXA_API_KEY"] = ""

# or colab secrets:

# from google.colab import userdata
# os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
# os.environ["EXA_API_KEY"] = userdata.get("EXA_API_KEY")

## Model Settings
Define a model to use, as we are going to use tool calling you need a capable model like `Kimi-K2` or `GLM`

In [6]:
class Console(RichConsole):
    def md(self, text):
        return self.print(Markdown(text))

console = Console()

MODEL_SMALL = "Qwen/Qwen3-235B-A22B-Instruct-2507"
MODEL_MEDIUM = "zai-org/GLM-4.5"
MODEL_LARGE = "moonshotai/Kimi-K2-Instruct"

WANDB_ENTITY = "team-prakash"
WANDB_PROJECT = "london-workshop-2025"

oai_client = openai.OpenAI(
    base_url='https://api.inference.wandb.ai/v1',
    api_key=os.getenv("WANDB_API_KEY"),
    project=f"{WANDB_ENTITY}/{WANDB_PROJECT}")
exa_client = Exa(api_key=os.getenv("EXA_API_KEY"))

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [7]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

weave: wandb version 0.22.3 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: Logged in as Weights & Biases user: prkshverma09.
weave: View Weave data at https://wandb.ai/team-prakash/london-workshop-2025/weave


## 1. Basic LLM Call with OpenAI SDK

Let's start with a simple call to the LLM using/

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/01_trace.png?raw=1)

In [8]:
# Define a simple message list (conversation history)
messages = [{"role": "user", "content": "Hello, LLM! How does an AI agent work?"}]

# Make the call
response = resp = oai_client.chat.completions.create(
    model = MODEL_SMALL,
    messages=messages,
    max_tokens=400,
)
# Print the response content
assistant_response = response.choices[0].message.content
console.md(f"LLM Response:\\n{assistant_response}")

# Click on the 🍩 linke below to see the trace in Weave 👇

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4e87-a2b6-70f0-9be5-9661d16ca5a6


LLM Response:\nHello! Great question! 😊 An AI agent is a software system that perceives its environment, makes    
decisions, and takes actions to achieve specific goals. Think of it as a "digital assistant" that can act          
autonomously, sometimes even learning and improving over time.                                                     

Here’s a breakdown of how an AI agent works:                                                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                 🔍 1. Perception                                                  

The agent gathers information from its environment using sensors or data inputs. These could be:                   

 • Text input (like your message to me)                                                                            
 • Camera images (for a self-driving car)                                                                          
 • Sensor data (temperature, GPS, etc.)                                                                            
 • Web APIs, databases, or user interactions                                                                       

👉 Example: A chatbot reads your message; a robot vacuum senses obstacles.                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                   🧠 2. Decision-Making (Reasoning & Planning)                                    

Using the input, the agent processes the information and decides what to do. This involves:                        

 • Rule-based logic (if X happens, do Y)                                                                           
 • Machine learning models (e.g., predicting the best action)                                                      
 • Search algorithms (planning a path)                                                                             
 • Large Language Models (LLMs) like me, to generate responses                                                     

👉 Example: A recommendation agent analyzes your past behavior to suggest a movie.                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   🤖 3. Action                                                    

The agent performs an action using actuators or output mechanisms:                                                 

 • Sending a message (like this one!)                                                                              
 • Moving a robot arm                                                                                              
 • Adjusting thermostat settings                                                                                   
 • Clicking a button in a browser (in web automation)                                                              

👉 Example: After deciding, a delivery drone changes course to avoid rain.                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                 🔁 4. Learning & Feedback (Optional but powerful)                                 

Many advanced agents learn from experience:                                                                        

 • Reinforcement Learning: Gets rewards/punishments for actions                                                    
 • Supervised Learning: Learns from labeled examples                                                               
 • Self-reflection: Some agents review

Because we imported `weave` and called `weave.init()`, the OpenAI SDK call above was automatically traced. You can open your Weave dashboard and see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [9]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.

@weave.op
def call_model(model_name: str, messages: list[dict[str, Any]], **kwargs) -> str:
    "Call a model with the given messages and kwargs."
    response = oai_client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=400,
        **kwargs
    )

    return response.choices[0].message

response = call_model(model_name=MODEL_SMALL, messages=messages)
console.md(response.content)

weave: retry_attempt
weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4e96-4ec5-7a6a-9cff-a38c2445d77a


Hello! Great question! 😊                                                                                          

An AI agent is a software system that perceives its environment through sensors (data inputs), processes that      
information using AI techniques, and takes actions to achieve specific goals. Think of it like a digital "robot"   
that can make decisions and act autonomously.                                                                      

Here’s a breakdown of how an AI agent works:                                                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   1. Perception                                                   

The agent gathers information from its environment using:                                                          

 • Sensors (in physical robots: cameras, microphones, etc.)                                                        
 • Data inputs (in software: user inputs, databases, APIs, files)                                                  

👉 Example: A chatbot reads your message; a self-driving car sees traffic via cameras.                             

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                          2. Processing & Decision-Making                                          

The agent uses AI models (like machine learning, deep learning, or rule-based systems) to:                         

 • Understand the input (e.g., natural language processing)                                                        
 • Evaluate possible actions                                                                                       
 • Decide on the best course of action based on goals                                                              

🧠 This can involve:                                                                                               

 • Planning (what steps to take)                                                                                   
 • Learning (improving from past experiences)                                                                      
 • Reasoning (logical inference)                                                                                   

👉 Example: A recommendation agent analyzes your past behavior to suggest a movie.                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     3. Action                                                     

The agent performs actions using:                                                                                  

 • Actuators (in robots: motors, speakers)                                                                         
 • Output mechanisms (in software: sending a message, updating a database)                                         

👉 Example: A smart thermostat adjusts the temperature; a chatbot replies to your question.                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                 4. Feedback Loop                                                  

Many agents learn from the outcomes of their actions:                                                              

 • They receive feedback (e.g., user ratings, success/failure signals)                                             
 • They update their models (via reinforcement learning, for example)                                              

🔁 This creates a cycle: Perceive → Think → Act → Learn                                                            

─────────────────────────────────────────

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/02_nested_trace.png?raw=1)


## 2. Introducing Tool Calling

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.

Check the Mistral docs for function calling: https://platform.openai.com/docs/guides/function-calling

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/function-calling-diagram-steps.png?raw=1)

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [10]:
@weave.op
def add_numbers(a: int, b: int) -> int:
    """Use this tool to add numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [12]:
add_numbers(1, 2)

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4e9b-fd38-7e0b-8dd5-966e1b0f7f2e


3

In [13]:
# this doesn't work...
call_model(model_name=MODEL_SMALL, messages=messages, tools=[add_numbers])

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4eab-ca4d-7ffc-9e3f-2d8e493d5e5f


TypeError: Object of type function is not JSON serializable

> We need to manually create the JSON schema describing this tool in a format that models *Mistral* understand.

In [14]:
# Manually define the tool schema
tool_add_numbers_schema = {
    "type": "function",
    "function": {
        "name": "add_numbers",
        "description": "Adds two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "The first number."
                },
                "b": {
                    "type": "integer",
                    "description": "The second number."
                }
            },
            "required": ["a", "b"]
        }
    }
}

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [46]:
messages = [
    {"role": "system", "content": "You are a helpful assistant use tools to answer questions."},
    {"role": "user", "content": "My lucky numbers are 77 and 11. What is their sum?"}]
response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[tool_add_numbers_schema])
console.print(response)

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4ed2-a7a3-7306-a446-28c62ebb02e4


ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-efc76cfd206a496ea74c993488f5acf4',
            function=Function(arguments='{"a": 77, "b": 11}', name='add_numbers'),
            type='function'
        )
    ],
    reasoning_content=None
)

## Manual Tool Call
The LLM's response might contain a request to call our tool (`response.choices[0].message.tool_calls`) or it might respond directly (`response.choices[0].message.content`). If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [16]:
if response.tool_calls:
    console.print("LLM requested a tool call:")
    for tool_call in response.tool_calls:
        function_name = tool_call.function.name
        function_args_str = tool_call.function.arguments
        function_args = json.loads(function_args_str)
        console.print(f"  - Tool: {function_name}, Args: {function_args_str}")
        if function_name == "add_numbers":
            tool_result_content = add_numbers(**function_args)

console.print(f"Final Result: {tool_result_content}")

LLM requested a tool call:

- Tool: add_numbers, Args: {"a": 77, "b": 11}

Final Result: 88

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4eb4-d4c5-753a-89d5-0e3c39cde578


We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [17]:
messages.append(response.model_dump())

In [18]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": str(tool_result_content)
})

In [19]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant use tools to answer questions.'},
 {'role': 'user',
  'content': 'My lucky numbers are 77 and 11. What is their sum?'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'chatcmpl-tool-af8d8a4c492340e1b4f2cbe1c5fdd2b0',
    'function': {'arguments': '{"a": 77, "b": 11}', 'name': 'add_numbers'},
    'type': 'function'}],
  'reasoning_content': None},
 {'tool_call_id': 'chatcmpl-tool-af8d8a4c492340e1b4f2cbe1c5fdd2b0',
  'role': 'tool',
  'name': 'add_numbers',
  'content': '88'}]

You should have a sequence of messages like this:


In [20]:
[m["role"] for m in messages]

['system', 'user', 'assistant', 'tool']

Now call the model again with the new messages and it will use the tool call result to answer the question

In [28]:
# messages[3]["content"] = '87'
# console.print(messages[3].get("content"))

87

In [30]:
console.print(messages[3].get("content"))
final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.print(final_response.content)

87

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4ec2-35b5-7ef7-8cfc-5ea2f22b0ffa


It looks like there was a mistake. Let me recalculate the sum of your lucky numbers, 77 and 11.

77 + 11 = 88

So, the correct sum is **88**.

## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [48]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a tool-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": inspect.getdoc(func).split("\\n")[0] if inspect.getdoc(func) else "",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }

    docstring = inspect.getdoc(func)
    param_descriptions = {}
    if docstring:
        args_section = False
        current_param = None
        for line in docstring.split('\\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped:
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" "): # Heuristic: end of args section
                     args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle list[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by list or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: OpenAI schema doesn't officially use default, but useful metadata

        schema["function"]["parameters"]["properties"][name] = param_schema
        if is_required:
            schema["function"]["parameters"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [49]:
tool_schema = generate_tool_schema(add_numbers)
console.print(tool_schema)

{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [50]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        console.print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [51]:
add_numbers = function_tool(add_numbers)
console.print(add_numbers.tool_schema)

{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

In [52]:
add_numbers.tool_schema

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': ''},
    'b': {'type': 'integer', 'description': ''}},
   'required': ['a', 'b']}}}

and call the tool =)

In [53]:
add_numbers(1, 2)

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4ed3-4602-7a2b-b6a1-ba0608ad7db0


3

### 3.1 Real Example using an API based tool

We are going to use the [EXA search API](https://docs.exa.ai/reference/getting-started).
- How does [EXA search works](https://docs.exa.ai/reference/how-exa-search-works#how-exa-search-works)
- Using exa search [as tool calling](https://docs.exa.ai/reference/openai-tool-calling)

In [54]:
query = "Recipes for cooking seabass?"

search_res = exa_client.search_and_contents(query=query, type='auto', num_results=4)

In [55]:
console.print(search_res)

SearchResponse(
    results=[
        Result(
            url='https://www.bbc.co.uk/food/sea_bass',
            id='https://www.bbc.co.uk/food/sea_bass',
            title='Sea bass recipes',
            score=None,
            published_date='2025-03-21T00:00:00.000Z',
            author='',
            image='',
            favicon=None,
            subpages=None,
            extras=None,
            text="[Close menu](https://www.bbc.co.uk/food/sea_bass)\n\n[Food](https://www.bbc.co.uk/food)\n\n# Sea 
bass recipes\n\nA superb sweet, white, textured fish, sea bass is a popular feature of menus but you can easily 
cook it at home. Our sea bass recipes make quick work or a whole fish or fillets.Try Nadiya's easy sea bass bake 
with potatoes, or add tarragon flavoured breadcrumbs as in Mary Berry's baked sea bass fillets.\n\n[Baked sea bass 
on potatoes](https://www.bbc.co.uk/food/recipes/whole_sea_bass_baked_on_19811)\n\nby José Pizarro\n\nWhole sea bass
is roasted on a layer of potatoes with herbs and a punchy dressing, served with a flavour-packed pickled walnut 
salad. This is a really easy way to make a simple fish dish a spectacular centrepiece.\n\nMain course\n\n## More 
sea bass recipes\n\n[Featured](https://www.bbc.co.uk/food/sea_bass#featured-content) 
[All](https://www.bbc.co.uk/food/sea_bass/a-z#featured-content)\n\n[**Pan-fried sea bass in harissa butter** by 
Georgina Hayden\\\n\\\nMain course](https://www.bbc.co.uk/food/recipes/pan-fried_sea_bass_in_88540)\n\n[**Tarragon 
crusted sea bass** by Mary Berry\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/tarragon_crusted_sea_93411)\n\n[**Sea bass with tomato, olive and herb 
sauce** by Mitch Tonks\\\n\\\nMain course](https://www.bbc.co.uk/food/recipes/seabasswithtomatooli_72150)\n\n[**Sea
bass with herb crust, pepper and balsamic dressing** by Bryn Williams\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/sea_bass_herb_crust_13119)\n\n[**Cured sea bass with purple sprouting 
broccoli, hazelnut pesto and bagna cauda** by Adam Byatt\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/cured_sea_bass_with_28896)\n\n[**Whole roasted sea bass with romesco 
sauce and potatoes** by Matt Tebbutt\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/roasted_sea_bass_romesco_06487)\n\n[**Whole baked sea bass** by Angela 
Hartnett\\\n\\\nMain course](https://www.bbc.co.uk/food/recipes/whole_baked_sea_bass_88065)\n\n[**Baked sea bass on
potatoes** by José Pizarro\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/whole_sea_bass_baked_on_19811)\n\n[**Mexican fish with refried beans** 
by Rick Stein\\\n\\\nMain course](https://www.bbc.co.uk/food/recipes/butterfield_grilled_bass_01988)\n\n[**Baked 
sea bass with fennel and porcini mushroom** by Theo Randall\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/baked_seabass_with_96694)\n\n[**Bouillabaisse** by Mary 
Berry\\\n\\\nMain course](https://www.bbc.co.uk/food/recipes/bouillabaisse_00342)\n\n[**Harissa baked sea bass with
baby potatoes, mint salsa and Mediterranean salad** by Donal Skehan\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/harissa_baked_sea_bass_91056)\n\n[**Sicilian-style fish and couscous** 
by Matt Tebbutt\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/sicilian-style_fish_and_07938)\n\n[**Pan-roasted sea bass with basil 
butter sauce** by Matt Tebbutt\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/wild_sea_bass_with_basil_25408)\n\n[**Sweet and sour fish with Sicilian 
orange salad** by Matt Tebbutt\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/sweet_and_sour_fish_with_73513)\n\n[**Yunnan-style grilled sea bass with
twice-cooked pork** by Ching-He Huang\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/grilled_sea_bass_with_96195)\n\n[**Salt-baked sea bass with beurre 
blanc** by Richard Bainbridge\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/salt-baked_bream_with_78937)\n\n[**Roasted sea bass with wild 
mushrooms** by Matt Teb

Let's explore the payload

In [56]:
console.md("\n-------------------\n".join(result.text for result in search_res.results))

]8;id=338121;https://www.bbc.co.uk/food/sea_bass\Close menu]8;;\                                                                                                         

]8;id=719763;https://www.bbc.co.uk/food\Food]8;;\                                                                                                               

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                Sea bass recipes                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

A superb sweet, white, textured fish, sea bass is a popular feature of menus but you can easily cook it at home.   
Our sea bass recipes make quick work or a whole fish or fillets.Try Nadiya's easy sea bass bake with potatoes, or  
add tarragon flavoured breadcrumbs as in Mary Berry's baked sea bass fillets.                                      

]8;id=296417;https://www.bbc.co.uk/food/recipes/whole_sea_bass_baked_on_19811\Baked sea bass on potatoes]8;;\                                                                                         

by José Pizarro                                                                                                    

Whole sea bass is roasted on a layer of potatoes with herbs and a punchy dressing, served with a flavour-packed    
pickled walnut salad. This is a really easy way to make a simple fish dish a spectacular centrepiece.              

Main course                                                                                                        


                                               More sea bass recipes                                               

]8;id=148329;https://www.bbc.co.uk/food/sea_bass#featured-content\Featured]8;;\ ]8;id=734856;https://www.bbc.co.uk/food/sea_bass/a-z#featured-content\All]8;;\                                                                                                       

]8;id=140242;https://www.bbc.co.uk/food/recipes/pan-fried_sea_bass_in_88540\Pan-fried sea bass in harissa butter]8;;\]8;id=653405;https://www.bbc.co.uk/food/recipes/pan-fried_sea_bass_in_88540\ by Georgina Hayden]8;;\                                                            
                                                                                                                   
]8;id=653405;https://www.bbc.co.uk/food/recipes/pan-fried_sea_bass_in_88540\Main course]8;;\                                                                                                        

]8;id=360242;https://www.bbc.co.uk/food/recipes/tarragon_crusted_sea_93411\Tarragon crusted sea bass]8;;\]8;id=574306;https://www.bbc.co.uk/food/recipes/tarragon_crusted_sea_93411\ by Mary Berry]8;;\                                                                            
                                                                                                                   
]8;id=574306;https://www.bbc.co.uk/food/recipes/tarragon_crusted_sea_93411\Main course]8;;\                                                                                                        

]8;id=908477;https://www.bbc.co.uk/food/recipes/seabasswithtomatooli_72150\Sea bass with tomato, olive and herb sauce]8;;\]8;id=243905;https://www.bbc.co.uk/food/recipes/seabasswithtomatooli_72150\ by Mitch Tonks]8;;\                                                          
                                                                                                                   
]8;id=243905;https://www.bbc.co.uk/food/recipes/seabasswithtomatooli_72150\Main course]8;;\                                                                                                        

]8;id=899331;https://www.bbc.co.uk/food/recipes/sea_bass_herb_crust_13119\Sea bass with herb 

In [58]:
@weave.op
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.

    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.

    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.

    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)

    output = []
    for result in search_results.results:
        output.append(
            {"title": result.title,
            "text": result.text,
            "url": result.url
            }
        )
    return output



In [59]:
exa_search.tool_schema

{'type': 'function',
 'function': {'name': 'exa_search',
  'description': 'Perform a search query on the web and retrieve the most relevant URLs and web content.\n\nThis function uses the Exa search API to find relevant web pages based on the query\nand returns their titles, text content, and URLs.\n\nArgs:\n    query: The search query. Use detailed, specific queries for better results.\n           The quality of results depends on the specificity of the query.\n    num_results: The number of search results to retrieve. Defaults to 5.\n\nReturns:\n    A list of dictionaries, each containing:\n        - title: The title of the web page\n        - text: The text content of the web page\n        - url: The URL of the web page',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string', 'description': ''},
    'num_results': {'type': 'integer', 'description': '', 'default': 5}},
   'required': ['query']}}}

We get a list of results with the relevant metadata.

In [60]:
search_results = exa_search("How do I cook seabass?")
console.print(search_results)

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4ed3-e089-7fd0-beb8-cec0253d2bc0


[
    {
        'title': "Chilean Sea Bass Recipe {Restaurant Style} - The Big Man's World ®",
        'text': '[Skip to content](https://thebigmansworld.com/thebigmansworld.com#main-content)\n\n# Chilean Sea 
Bass Recipe\n\nBy [Arman Liew](https://thebigmansworld.com/arman-liew/)published on May 20, 2025\n\n[Jump to Recipe
▼](https://thebigmansworld.com/chilean-sea-bass-recipe/#wprm-recipe-container-61837)\n\nReader Rating\n\nTotal Time
10 minutesmins\n\nServings 4servings\n\n_This post may contain affiliate links. See my [disclosure 
policy](https://thebigmansworld.com/privacy-policy/)._\n\n_This pan seared **Chilean sea bass recipe** comes 
together in under 10 minutes and cooks perfectly every single time! It’s easy, elegant, and delicious._\n\n**★★★★★ 
REVIEW**\n\n_“OMG, this recipe was the easiest and the most delicious! My sister had never had sea bass before, so 
I made it for both of us. I did get it without the skin, and it was absolutely amazing! If you haven’t tried this, 
please use this recipe, you won’t regret it, the absolute best!”_ – Pam\n\nTable of Contents\n\n1. [Key 
Ingredients](https://thebigmansworld.com/thebigmansworld.com#h-key-ingredients)\n2. [How to cook Chilean sea 
bass](https://thebigmansworld.com/thebigmansworld.com#h-how-to-cook-chilean-sea-bass)\n3. [Chilean sea bass cooking
tips](https://thebigmansworld.com/thebigmansworld.com#h-chilean-sea-bass-cooking-tips)\n4. [How to 
serve](https://thebigmansworld.com/thebigmansworld.com#h-how-to-serve)\n5. [Seared Chilean Sea Bass Recipe (Recipe 
Card)](https://thebigmansworld.com/thebigmansworld.com#wprm-recipe-container-61837)\n\nFriends, if you are looking 
for a simple fish dinner that’s elegant enough for date night, a dinner party, or a special occasion, my quick, 
pan-seared Chilean sea bass recipe is a must-try. While it’s a popular choice at fine dining restaurants or elegant
events, making it at home is SO easy and tastes just as good- if not better!\n\nIt’s so tender, flaky, and 
naturally buttery that the sea bass practically melts in your mouth. I love that it takes just ten minutes to make 
(prep AND cook time included) and has minimal clean-up. It’s drizzled with a simple garlic lemon butter, which lets
the natural flavor of the Chilean sea bass shine.\n\nIf you love elegant seafood dinners, try my **[parmesan 
crusted tilapia](https://thebigmansworld.com/parmesan-crusted-tilapia/)**, **[grilled tuna 
steak](https://thebigmansworld.com/grilled-tuna-steak/)**, **[roasted 
branzino](https://thebigmansworld.com/branzino-recipe/)**, and 
**[rockfish](https://thebigmansworld.com/pan-seared-rockfish-recipe/)** next.\n\nWhat is Chilean Sea 
Bass?\n\nChilean sea bass (also called Patagonian toothfish, mero, or icefish), is a deep-water species native to 
the cold sub-Antarctic regions of the Pacific, southern Atlantic, and Indian Oceans.\n\nIt’s high in protein and 
super low in carbs. It’s famous for its silky, buttery texture and mild flavor (no fishy taste). It’s considered a 
premium fish due as its hard to overcook and super versatile.\n\n## Key Ingredients\n\n- **Chilean sea bass 
fillets**. Either fresh or frozen fillets.\xa0Because of the slightly higher price tag, I prefer to get the fillets
from a fishmonger over a grocery store. When choosing sea bass, opt for fillets that have no strong odors, no dry 
spots, and are firm to touch.\n- **Kosher salt and black pepper**. To taste.\n- **Unsalted butter**. Pan-frying 
white fish fillets in butter takes the dish’s flavor to another level and cooks the fish perfectly. Olive oil will 
also work.\n- **Garlic**. Freshly minced garlic, not the jarred kind!\n- **Lemon**. Fresh lemons to squeeze over 
the freshly cooked fish.\n\n## How to cook Chilean sea bass\n\n_This is an overview with step-by-step photos. Full 
ingredients & instructions are in the [recipe card 
below](https://thebigmansworld.com/chilean-sea-bass-recipe/#wprm-recipe-container-61837)._\n\n**Step 1-** Pat dry 
the fish filets wi

In [61]:
messages = [
    {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
    {"role": "user", "content": "How do I cook seabass?"}]

response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[exa_search.tool_schema])
console.print(response)

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4ed4-27ae-7756-aaad-e47a21f5a8b0


ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-ca013782678d43058070d1aa88f69e7a',
            function=Function(
                arguments='{"query": "how to cook seabass step by step", "num_results": 5}',
                name='exa_search'
            ),
            type='function'
        )
    ],
    reasoning_content=None
)

Let's create some helper functions to perform the tool calls

In [62]:
def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], tool_calls: list[Callable]) -> list[dict]:
    "Perform the tool calls and return the messages with the tool call results"
    messages = []
    if not tool_calls:
        return messages
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        with console.status(f"[bold cyan]Executing {function_name}...[/bold cyan]"):
            tool = get_tool(tools, function_name)
            tool_response = tool(**function_args) # doesn't handle async

        # Create panel content
        panel_content = f"[bold cyan]🔧 Tool Call:[/bold cyan] {function_name}\n\n"
        panel_content += f"[dim]Args: {tool_call.function.arguments}[/dim]\n\n"

        if isinstance(tool_response, list):
            panel_content += f"[green]✓[/green] Found {len(tool_response)} results"
        else:
            panel_content += f"[green]✓[/green] {function_name} executed successfully"

        console.print(Panel(panel_content, border_style="cyan"))

        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "content": str(tool_response),
        })
    return messages

In [63]:

# add the tool call result to the messages
messages.append(response.model_dump())
console.print(messages)
messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
console.print(messages)
messages.append({
    "role": "user",
    "content": "Answer my previous query based on the search results.",})
console.print(messages)

final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.rule("Final Model Response")
console.md(final_response.content)


[
    {
        'role': 'system',
        'content': 'You are an agent that has access to an advanced search engine. Please provide the user with the
information they are looking for by using the search tool provided. Make sure to keep the sources. Return in 
markdown format.'
    },
    {'role': 'user', 'content': 'How do I cook seabass?'},
    {
        'content': None,
        'refusal': None,
        'role': 'assistant',
        'annotations': None,
        'audio': None,
        'function_call': None,
        'tool_calls': [
            {
                'id': 'chatcmpl-tool-ca013782678d43058070d1aa88f69e7a',
                'function': {
                    'arguments': '{"query": "how to cook seabass step by step", "num_results": 5}',
                    'name': 'exa_search'
                },
                'type': 'function'
            }
        ],
        'reasoning_content': None
    }
]

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4ed4-9d4e-7f3b-a2bd-1804ab903ddb


Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "how to cook seabass step by step", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[
    {
        'role': 'system',
        'content': 'You are an agent that has access to an advanced search engine. Please provide the user with the
information they are looking for by using the search tool provided. Make sure to keep the sources. Return in 
markdown format.'
    },
    {'role': 'user', 'content': 'How do I cook seabass?'},
    {
        'content': None,
        'refusal': None,
        'role': 'assistant',
        'annotations': None,
        'audio': None,
        'function_call': None,
        'tool_calls': [
            {
                'id': 'chatcmpl-tool-ca013782678d43058070d1aa88f69e7a',
                'function': {
                    'arguments': '{"query": "how to cook seabass step by step", "num_results": 5}',
                    'name': 'exa_search'
                },
                'type': 'function'
            }
        ],
        'reasoning_content': None
    },
    {
        'tool_call_id': 'chatcmpl-tool-ca013782678d43058070d1aa88f69e7a',
        'role': 'tool',
        'content': '[{\'title\': "Chilean Sea Bass Recipe {Restaurant Style} - The Big Man\'s World ®", \'text\': 
\'[Skip to content](https://thebigmansworld.com/thebigmansworld.com#main-content)\\n\\n# Chilean Sea Bass 
Recipe\\n\\nBy [Arman Liew](https://thebigmansworld.com/arman-liew/)published on May 20, 2025\\n\\n[Jump to Recipe 
▼](https://thebigmansworld.com/chilean-sea-bass-recipe/#wprm-recipe-container-61837)\\n\\nReader Rating\\n\\nTotal 
Time 10 minutesmins\\n\\nServings 4servings\\n\\n_This post may contain affiliate links. See my [disclosure 
policy](https://thebigmansworld.com/privacy-policy/)._\\n\\n_This pan seared **Chilean sea bass recipe** comes 
together in under 10 minutes and cooks perfectly every single time! It’s easy, elegant, and 
delicious._\\n\\n**★★★★★ REVIEW**\\n\\n_“OMG, this recipe was the easiest and the most delicious! My sister had 
never had sea bass before, so I made it for both of us. I did get it without the skin, and it was absolutely 
amazing! If you haven’t tried this, please use this recipe, you won’t regret it, the absolute best!”_ – 
Pam\\n\\nTable of Contents\\n\\n1. [Key 
Ingredients](https://thebigmansworld.com/thebigmansworld.com#h-key-ingredients)\\n2. [How to cook Chilean sea 
bass](https://thebigmansworld.com/thebigmansworld.com#h-how-to-cook-chilean-sea-bass)\\n3. [Chilean sea bass 
cooking tips](https://thebigmansworld.com/thebigmansworld.com#h-chilean-sea-bass-cooking-tips)\\n4. [How to 
serve](https://thebigmansworld.com/thebigmansworld.com#h-how-to-serve)\\n5. [Seared Chilean Sea Bass Recipe (Recipe
Card)](https://thebigmansworld.com/thebigmansworld.com#wprm-recipe-container-61837)\\n\\nFriends, if you are 
looking for a simple fish dinner that’s elegant enough for date night, a dinner party, or a special occasion, my 
quick, pan-seared Chilean sea bass recipe is a must-try. While it’s a popular choice at fine dining restaurants or 
elegant events, making it at home is SO easy and tastes just as good- if not better!\\n\\nIt’s so tender, flaky, 
and naturally buttery that the sea bass practically melts in your mouth. I love that it takes just ten minutes to 
make (prep AND cook time included) and has minimal clean-up. It’s drizzled with a simple garlic lemon butter, which
lets the natural flavor of the Chilean sea bass shine.\\n\\nIf you love elegant seafood dinners, try my **[parmesan
crusted tilapia](https://thebigmansworld.com/parmesan-crusted-tilapia/)**, **[grilled tuna 
steak](https://thebigmansworld.com/grilled-tuna-steak/)**, **[roasted 
branzino](https://thebigmansworld.com/branzino-recipe/)**, and 
**[rockfish](https://thebigmansworld.com/pan-seared-rockfish-recipe/)** next.\\n\\nWhat is Chilean Sea 
Bass?\\n\\nChilean sea bass (also called Patagonian toothfish, mero, or icefish), is a deep-water species native to
the cold sub-Antarctic regions of the Pacific, southern Atlantic, and Indian Oceans.\\n\\nIt’s high in protein and 
super low in carbs. It’s famous for 

[
    {
        'role': 'system',
        'content': 'You are an agent that has access to an advanced search engine. Please provide the user with the
information they are looking for by using the search tool provided. Make sure to keep the sources. Return in 
markdown format.'
    },
    {'role': 'user', 'content': 'How do I cook seabass?'},
    {
        'content': None,
        'refusal': None,
        'role': 'assistant',
        'annotations': None,
        'audio': None,
        'function_call': None,
        'tool_calls': [
            {
                'id': 'chatcmpl-tool-ca013782678d43058070d1aa88f69e7a',
                'function': {
                    'arguments': '{"query": "how to cook seabass step by step", "num_results": 5}',
                    'name': 'exa_search'
                },
                'type': 'function'
            }
        ],
        'reasoning_content': None
    },
    {
        'tool_call_id': 'chatcmpl-tool-ca013782678d43058070d1aa88f69e7a',
        'role': 'tool',
        'content': '[{\'title\': "Chilean Sea Bass Recipe {Restaurant Style} - The Big Man\'s World ®", \'text\': 
\'[Skip to content](https://thebigmansworld.com/thebigmansworld.com#main-content)\\n\\n# Chilean Sea Bass 
Recipe\\n\\nBy [Arman Liew](https://thebigmansworld.com/arman-liew/)published on May 20, 2025\\n\\n[Jump to Recipe 
▼](https://thebigmansworld.com/chilean-sea-bass-recipe/#wprm-recipe-container-61837)\\n\\nReader Rating\\n\\nTotal 
Time 10 minutesmins\\n\\nServings 4servings\\n\\n_This post may contain affiliate links. See my [disclosure 
policy](https://thebigmansworld.com/privacy-policy/)._\\n\\n_This pan seared **Chilean sea bass recipe** comes 
together in under 10 minutes and cooks perfectly every single time! It’s easy, elegant, and 
delicious._\\n\\n**★★★★★ REVIEW**\\n\\n_“OMG, this recipe was the easiest and the most delicious! My sister had 
never had sea bass before, so I made it for both of us. I did get it without the skin, and it was absolutely 
amazing! If you haven’t tried this, please use this recipe, you won’t regret it, the absolute best!”_ – 
Pam\\n\\nTable of Contents\\n\\n1. [Key 
Ingredients](https://thebigmansworld.com/thebigmansworld.com#h-key-ingredients)\\n2. [How to cook Chilean sea 
bass](https://thebigmansworld.com/thebigmansworld.com#h-how-to-cook-chilean-sea-bass)\\n3. [Chilean sea bass 
cooking tips](https://thebigmansworld.com/thebigmansworld.com#h-chilean-sea-bass-cooking-tips)\\n4. [How to 
serve](https://thebigmansworld.com/thebigmansworld.com#h-how-to-serve)\\n5. [Seared Chilean Sea Bass Recipe (Recipe
Card)](https://thebigmansworld.com/thebigmansworld.com#wprm-recipe-container-61837)\\n\\nFriends, if you are 
looking for a simple fish dinner that’s elegant enough for date night, a dinner party, or a special occasion, my 
quick, pan-seared Chilean sea bass recipe is a must-try. While it’s a popular choice at fine dining restaurants or 
elegant events, making it at home is SO easy and tastes just as good- if not better!\\n\\nIt’s so tender, flaky, 
and naturally buttery that the sea bass practically melts in your mouth. I love that it takes just ten minutes to 
make (prep AND cook time included) and has minimal clean-up. It’s drizzled with a simple garlic lemon butter, which
lets the natural flavor of the Chilean sea bass shine.\\n\\nIf you love elegant seafood dinners, try my **[parmesan
crusted tilapia](https://thebigmansworld.com/parmesan-crusted-tilapia/)**, **[grilled tuna 
steak](https://thebigmansworld.com/grilled-tuna-steak/)**, **[roasted 
branzino](https://thebigmansworld.com/branzino-recipe/)**, and 
**[rockfish](https://thebigmansworld.com/pan-seared-rockfish-recipe/)** next.\\n\\nWhat is Chilean Sea 
Bass?\\n\\nChilean sea bass (also called Patagonian toothfish, mero, or icefish), is a deep-water species native to
the cold sub-Antarctic regions of the Pacific, southern Atlantic, and Indian Oceans.\\n\\nIt’s high in protein and 
super low in carbs. It’s famous for 

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4ed4-a30d-7a04-8690-3b9cc1477bda


────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

Cooking seabass is simple and can be done in multiple ways, including pan-searing, baking, grilling, or using an   
air fryer. Below are the most popular and effective methods based on the search results:                           

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                             1. Pan-Seared Chilean Sea Bass (Quick & Restaurant-Style)                             

This method delivers a crispy exterior and tender, flaky interior in under 15 minutes.                             

Ingredients:                                                                                                       

 • Sea bass fillets (skin-on recommended)                                                                          
 • Salt and black pepper                                                                                           
 • Butter or olive oil                                                                                             
 • Garlic (optional)                                                                                               
 • Lemon juice                                                                                                     

Steps:                                                                                                             

 1 Pat dry the fillets with a paper towel to remove moisture.                                                      
 2 Season both sides with salt and pepper.                                                                         
 3 Heat butter or oil in a skillet (preferably cast iron) over medium-high heat.                                   
 4 Place fillets skin-side down and sear for 3–5 minutes without moving to form a golden crust.                    
 5 Flip and cook the other side for 2–4 minutes until the internal temperature reaches 140–145°F (60–63°C).        
 6 Remove from heat, drizzle with lemon juice and optional garlic butter sauce.                                    

Tip: Don’t move the fish too soon—wait until it releases easily from the pan.                                      

Source: ]8;id=937828;https://thebigmansworld.com/chilean-sea-bass-recipe/\The Big Man's World]8;;\, ]8;id=302885;https://www.alphafoodie.com/chilean-sea-bass-recipe/\Alphafoodie]8;;\                                                                           

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                       2. Baked Sea Bass (Easy & Hands-Off)                                        

Perfect for a low-carb or keto-friendly meal.                                                                      

Ingredients:                                                                                                       

 • Sea bass fillets                                                                                                
 • Melted butter                                                                                                   
 • Salt, pepper, paprika (optional)                                                                                
 • Lemon                                                                                                           

Steps:                                                                                                             

 1 Preheat oven to **4

Let's wrap this in a function:

In [64]:
@weave.op
def research(query: str) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
        {"role": "user", "content": query}]

    # call model with tools
    response = call_model(
        model_name=MODEL_SMALL,
        messages=messages,
        tools=[exa_search.tool_schema])

    # add the response to the messages
    messages.append(response.model_dump())

    # perform the tool calls
    messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))

    # prompt the model to be grounded
    messages.append({"role": "user","content": "Answer my previous query based on the search results.",})

    final_response = call_model(model_name=MODEL_SMALL, messages=messages)
    return final_response.content

In [65]:
result = research("What are the most popular pokemons?")
console.md(result)

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4ed6-9b9a-7018-81f3-bd4399dfc60a


Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "most popular Pokémon of all time", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Based on the search results, the most popular Pokémon are determined by fan votes, cultural impact, and iconic     
status. Here are the top Pokémon recognized as the most popular:                                                   

                                                   1. Charizard                                                    

 • Ranking: #1 in a 2019 Reddit fan vote with 1,107 votes.                                                         
 • Reason: Known for its powerful design, fiery abilities, and popularity in both the games and the original       
   animated series. Its first-edition trading card is also one of the most valuable and iconic.                    

                                                     2. Gengar                                                     

 • Ranking: #2 in the same Reddit poll with 1,056 votes.                                                           
 • Reason: A fan-favorite Ghost-type Pokémon from Generation I, beloved for its eerie yet charming design. It has  
   gained even more popularity through Halloween-themed merchandise and its role in battles.                       

                                                    3. Arcanine                                                    

 • Ranking: #3 with 923 votes.                                                                                     
 • Reason: Celebrated for its majestic, lupine appearance and legendary speed (said to run over 6,200 miles in a   
   day). It's often praised for its design and strength.                                                           

                                                   4. Bulbasaur                                                    

 • Ranking: #4 with 710 votes.                                                                                     
 • Reason: One of the most popular starter Pokémon, especially among fans of the original 151. It gained prominence
   through Ash’s team in the anime and is often seen as a symbol of the franchise’s beginning.                     

                                                Honorable Mentions:                                                

 • Lucario (#7) – Popular for its role in the anime, movies, and Super Smash Bros. series.                         
 • Eevee (#9) – Loved for its evolution versatility and cute design.                                               
 • Dragonite (#10) – A fan-favorite for its powerful stats and gentle personality despite its size.                
 • Pikachu – While not in the top 10 of the Reddit

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/04_pokedex.png?raw=1)

This is "Almost" an agent, but it's missing the loop. Let's add that next.

## 4. Implementing a Basic Agentic Loop

Let's implement a basic agentic loop. We'll use the `pokedex` function we just created. The implementation we have above has some limitations:
- Its a single turn, so if it fails to answer my question in one pass it is over.

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/05_agent.png?raw=1)

From the really good [Anthropic Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article and encourage people to read it.

A simple for loop

In [66]:
@weave.op
def research_loop(query: str, max_turns: int = 4, tools = [exa_search, add_numbers]) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format."},
        {"role": "user", "content": query}]

    for turn in range(max_turns):
        console.rule(f"Agent Loop Turn {turn + 1}/{max_turns}")

        # call model with tools
        response = call_model(
            model_name=MODEL_MEDIUM,
            messages=messages,
            tools=[t.tool_schema for t in tools])

        # add the response to the messages
        messages.append(response.model_dump())

        # if the LLM requested tool calls, perform them
        if response.tool_calls:
            # perform the tool calls
            tool_outputs = perform_tool_calls(tools=tools, tool_calls=response.tool_calls)
            messages.extend(tool_outputs)
        # LLM gave content response
        elif response.content:
            console.rule("Final Model Response")
            console.md(response.content)
            return response.content
        else:
            print("LLM response had neither content nor tool calls. Stopping loop.")
            break

In [67]:
res = research_loop("What is the sum of the populations of the 2 major EU cities?")

─────────────────────────────────────────────── Agent Loop Turn 1/4 ───────────────────────────────────────────────

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4ed8-657c-749c-9c9d-4f48539629c8


Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "largest cities European Union by population 2024", "num_results": 5}                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 2/4 ───────────────────────────────────────────────

────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

From the search results, I can see there are different definitions of "major cities" based on different criteria:  

 1 City proper/administrative boundaries (from Wikipedia):                                                         
    • Berlin: 3,897,145 population (as of 31 Dec 2024)                                                             
    • Madrid: 3,460,771 population (as of 1 Jan 2024)                                                              
    • Rome: 2,746,984 population (as of 1 Jan 2025)                                                                
    • Paris: 2,048,472 population (as of 1 Jan 2025)                                                               
 2 Metropolitan areas/urban areas (from World Atlas and World Population Review):                                  
    • Moscow, Russia: ~12.7 million                                                                                
    • Paris, France: ~11.2-11.3 million                                                                            
    • London, UK: ~9.6-9.8 million                                                                                 
    • Madrid, Spain: ~6.7-6.8 million                                                                              
 3 EU Metropolitan regions (from Eurostat):                                                                        
    • Paris: 12.4 million (2023)                                                                                   
    • Madrid: 6.9 million (2023)                                                                                   
    • Barcelona: 5.8 million (2023)                                                                                
    • Berlin: 5.5 million (2023)                                                                                   
    • Ruhrgebiet: 5.1 million (2023)                                                                               

The issue is that "major cities" can be defined in different ways:                                                 

 • City proper (administrative boundaries)                                                                         
 • Urban area/metropolitan area                                                                                    
 • Metropolitan regions                                                                                            

For the EU specifically, I should focus on cities that are actually within the European Union. Russia (Moscow, St. 
Petersburg) and the UK (London) are not in the EU, so they shouldn't be counted.                                   

Looking at the EU data from Eurostat, the largest EU metropolitan regions are:                                     

 1 Paris: 12.4 million                                                                                             
 2 Madrid: 6.

# 5. Structuring the Agent with Classes

The loop above works, but for more complex agents, encapsulating the logic and state within classes is much better. We'll define:
- `AgentState`: A Pydantic model to hold the conversation history and potentially other state.
- `SimpleAgent`: A class containing the agent's configuration (model, system message, tools) and logic (`step`, `run`).

In [68]:
class AgentState(BaseModel):
    """Manages the state of the agent."""
    messages: list[dict[str, Any]] = Field(default_factory=list)
    step: int = Field(default=0)
    final_assistant_content: str | None = None # Populated at the end of a run

In [69]:
class SimpleAgent:
    """A simple agent class with tracing, state, and tool processing."""
    def __init__(self, model_name: str, system_message: str, tools: list[Callable], state_class: BaseModel = AgentState):
        self.model_name = model_name
        self.system_message = system_message
        self.tools = [function_tool(t) for t in tools] # add schemas to the tools
        self.state_class = state_class

    @weave.op(name="SimpleAgent.step") # Trace each step
    def step(self, state: AgentState) -> AgentState:
        step = state.step + 1
        messages = state.messages
        final_assistant_content = None
        try:
            # call model with tools
            response = call_model(
                model_name=self.model_name,
                messages=messages,
                tools=[t.tool_schema for t in self.tools])

            # add the response to the messages
            messages.append(response.model_dump())

            # if the LLM requested tool calls, perform them
            if response.tool_calls:
                # perform the tool calls
                tool_outputs = perform_tool_calls(tools=self.tools, tool_calls=response.tool_calls)
                messages.extend(tool_outputs)

            # LLM gave content response
            else:
                final_assistant_content = response.content
        except Exception as e:
            console.print(f"ERROR in Agent Step: {e}")
            # Add an error message to history to indicate failure
            messages.append({"role": "assistant", "content": f"Agent error in step: {str(e)}"})
            final_assistant_content = f"Agent error in step {step}: {str(e)}"
        return self.state_class(messages=messages, step=step, final_assistant_content=final_assistant_content)

    @weave.op(name="SimpleAgent.run")
    def run(self, user_prompt: str, max_turns: int = 10, **state_kwargs: Any) -> AgentState:
        state = self.state_class(messages=[
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": user_prompt}], **state_kwargs)
        for _ in range(max_turns):
            console.rule(f"Agent Loop Turn {state.step+1}/{max_turns}")
            state = self.step(state)
            if state.final_assistant_content:
                return state
        return state



![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/07_simple_agent.png?raw=1)

In [70]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search, add_numbers],
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?", max_turns=3)
print(f"Final response: {state.final_assistant_content}")


─────────────────────────────────────────────── Agent Loop Turn 1/3 ───────────────────────────────────────────────

weave: retry_attempt
weave: retry_attempt


Output()

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4ee2-2dd6-7ac7-b752-3381594fe372


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "Ash Ketchum first three Pokémon weights", "num_results": 5}                                    │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 2/3 ───────────────────────────────────────────────

Final response: The first three Pokémon that Ash Ketchum caught in the Pokémon anime are:

1. **Pikachu**  
2. **Caterpie** (which evolved into **Butterfree**)  
3. **Pidgeotto**

Now, let's find their weights using reliable Pokémon data:

### 1. **Pikachu**
- **Weight**: 13.2 lbs (6.0 kg)  
(Source: [Pokémon DB - Pikachu](https://pokemondb.net/pokedex/pikachu))

### 2. **Butterfree** (evolved from Caterpie)
- **Weight**: 70.5 lbs (32.0 kg)  
(Source: [Pokémon DB - Butterfree](https://pokemondb.net/pokedex/butterfree))

### 3. **Pidgeotto**
- **Weight**: 66.1 lbs (30.0 kg)  
(Source: [Pokémon DB - Pidgeotto](https://pokemondb.net/pokedex/pidgeotto))

---

### Combined Weight:
- **In pounds (lbs)**:  
  13.2 (Pikachu) + 70.5 (Butterfree) + 66.1 (Pidgeotto) = **149.8 lbs**

- **In kilograms (kg)**:  
  6.0 + 32.0 + 30.0 = **68.0 kg**

---

### ✅ Final Answer:
The combined weight of Ash's first three Pokémon is **149.8 lbs (68.0 kg)**.


In [71]:
@weave.op
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search_and_refine(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.

    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.

    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.

    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)

    @weave.op
    def refine_search_result(result, query):
        messages = [
            {"role":"system", "content": f"Your task is to extract from the search results only the info that is relevant to answer the query"},
            {"role": "user", "content": f"- query: {query}\n- Search result: {result}"}
        ]
        refined_search = call_model(model_name=MODEL_SMALL, messages=messages)
        return refined_search.content

    output = []
    for item, result in enumerate(search_results.results):
        console.print(f"Refining result {item+1}")
        refined_text = refine_search_result(result.text, query)
        output.append(
            {"title": result.title,
            "text": refined_text,
            "url": result.url
            }
        )
    return output

In [73]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search_and_refine, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?", max_turns=3)
print(f"Final response: {state.final_assistant_content}")

weave: 🍩 https://wandb.ai/team-prakash/london-workshop-2025/r/call/019a4f04-3591-7833-a521-114597260c7a


─────────────────────────────────────────────── Agent Loop Turn 1/3 ───────────────────────────────────────────────

Output()

Refining result 1

Refining result 2

...

Refining result 4

Refining result 5

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search_and_refine                                                                             │
│                                                                                                                 │
│ Args: {"query": "Ash's first three Pokémon names and their weights", "num_results": 5}                          │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 2/3 ───────────────────────────────────────────────

Final response: Ash's first three Pokémon, based on the order he acquired them in the Pokémon anime series, are:

1. **Pikachu** – Weight: **6.0 kg**  
2. **Caterpie** – Weight: **2.9 kg**  
3. **Pidgeotto** – Weight: **30.0 kg**  

> *Note: While Pikachu was the first Pokémon Ash received, Caterpie was the first wild Pokémon he caught, and Pidgeotto was his third catch. The order is based on the sequence of capture during the Kanto journey.*

### Combined Weight:
$$
6.0\, \text{kg} + 2.9\, \text{kg} + 30.0\, \text{kg} = \boxed{38.9\, \text{kg}}
$$

**Total combined weight: 38.9 kilograms**

*Source: [Game Rant - Pokémon: Ash's First 10 Pokémon, Ranked](https://gamerant.com/pokemon-ashs-first-pokemon-ranked/)*


Possible improvements to the SimpleAgent:
- Give the model info about the state of the conversation, you could inject a message with the model context pressure, steps left, etc.
- Structured output. Make the model output a specific format, for instance a JSON with the expected fields.
- Add more tools like read and write files, access a database.
- Agent handoff: Agent1 does triage and Agent2 executes specific tasks.

## Evaluating our SimpleAgent

In [87]:
# Run this in colab

# !git clone https://github.com/morganmcg1/deep-research-bot/
!mv deep-research-bot/data /data

mv: cannot stat 'deep-research-bot/data': No such file or directory


In [82]:
import sys
from functools import partial
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [90]:
from deep_research_bot.evaluation.eval import run_evaluation
from deep_research_bot.evaluation.eval_config import EvalConfig

MAX_TURNS=5

agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search_and_refine, add_numbers],
)

eval_config = EvalConfig(
    evaluation_name=f"SimpleAgent_max-turns-{MAX_TURNS}_{agent.model_name.split('/')[-1]}",
    trials=2,
    limit=20,
    judge_model="deepseek-ai/DeepSeek-R1-0528",
    weave_parallelism=4,
    queries=project_root / "data/prompt_data/query.jsonl",
    reference=project_root / "data/test_data/cleaned_data/reference.jsonl",
    criteria=project_root / "data/criteria_data/criteria.jsonl",
)

results = await run_evaluation(
    eval_config=eval_config,
    agent_callable=partial(agent.run, max_turns=MAX_TURNS),  # <- partial to limit the number of agent turns
)
results

ValueError: OPENAI_API_KEY is not set, please set it in the .env file.